<a href="https://colab.research.google.com/github/vashadz/test/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import ipywidgets as widgets
from IPython.display import display

# Read data from uploaded CSV file
data = pd.read_csv('Earthquake Data.csv')

# Function to update heatmap based on user input
def update_heatmap(magnitude_threshold):
    filtered_data = data[data['mag'] >= magnitude_threshold]

    # Clear previous map
    if 'map' in globals():
        del map

    # Create a Folium map centered at (0, 0)
    map = folium.Map(location=[0, 0], zoom_start=2, tiles='OpenStreetMap')

    # Add HeatMap layer
    HeatMap(data=filtered_data[['latitude', 'longitude', 'mag']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(),
            radius=15, gradient={0.2: 'blue', 0.4: 'green', 0.6: 'orange', 1: 'red'}, max_val=data['mag'].max()).add_to(map)

    # Add circle markers with pop-up information and shaking effect
    for index, row in filtered_data.iterrows():
        radius = row['mag'] * 1  # Adjust the multiplier as needed to control the circle size based on magnitude
        shaking_intensity = radius * 2

        # Define the shaking effect with CSS animation and thinner outline
        circle_style = f"""
        radius: {radius}px;
        fillOpacity: 0.6;
        color: blue;
        fillColor: blue;
        fillOpacity: 0.3;
        weight: 0.0;  /* Outline thickness (set to 0.5 for a thinner outline) */
        opacity: 0.0; /* Outline opacity */
        cursor: pointer;
        """

         # Define the custom HTML for the pop-up with shaking effect
        popup_html = f"""
        <div style="animation: shake 0.5s {shaking_intensity}; cursor: pointer;">
            <strong>Magnitude: {row['mag']}</strong><br>
            Location: {row['place']}
        </div>
        <style>
            @keyframes shake {{
                 0% {{ transform: translate(1px, 1px) rotate(0deg); }}
            10% {{ transform: translate(-1px, -2px) rotate(-1deg); }}
            20% {{ transform: translate(-{shaking_intensity}px, 0px) rotate(1deg); }}
            30% {{ transform: translate({shaking_intensity}px, {shaking_intensity}px) rotate(0deg); }}
            40% {{ transform: translate(1px, -1px) rotate(1deg); }}
            50% {{ transform: translate(-1px, 2px) rotate(-1deg); }}
            60% {{ transform: translate(-{shaking_intensity}px, 1px) rotate(0deg); }}
            70% {{ transform: translate({shaking_intensity}px, 1px) rotate(-1deg); }}
            80% {{ transform: translate(1px, -1px) rotate(1deg); }}
            90% {{ transform: translate(-1px, 2px) rotate(0deg); }}
            100% {{ transform: translate(1px, -2px) rotate(-1deg); }}
        }}
        """

        # Create a circle marker with shaking effect
        circle_marker = folium.CircleMarker(location=[row['latitude'], row['longitude']],
                                            radius=radius,
                                            style=circle_style,
                                            popup=folium.Popup(folium.IFrame(popup_html, width=200, height=100), max_width=300))

        # Add the circle marker to the map
        circle_marker.add_to(map)

    # Display the updated map
    display(map)

# Create a slider widget for magnitude threshold
magnitude_slider = widgets.FloatSlider(value=4.0, min=3.0, max=9.0, step=0.1, description='Magnitude Threshold:')

# Create an interactive plot with the slider
interactive_plot = widgets.interactive(update_heatmap, magnitude_threshold=magnitude_slider)

# Display the interactive plot
display(interactive_plot)


interactive(children=(FloatSlider(value=4.0, description='Magnitude Threshold:', max=9.0, min=3.0), Output()),…